In [1]:
import pandas as pd #dataintables
import numpy as np #maths
import re #Regular Expressions
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

In [2]:
data = pd.read_csv('swiggy.csv')
print("Columns in the dataset:")
print(data.columns.tolist())
print(data.describe())

Columns in the dataset:
['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating', 'Food Item', 'Food Type', 'Delivery Time', 'Review']
               ID  Restaurant Price   Avg Rating  Total Rating
count  8000.00000       8000.000000  8000.000000   8000.000000
mean   4000.50000        544.587500     4.129900   4979.977500
std    2309.54541        287.968871     0.645791   2877.285148
min       1.00000        100.000000     3.000000     51.000000
25%    2000.75000        300.000000     3.500000   2476.000000
50%    4000.50000        500.000000     4.200000   4989.500000
75%    6000.25000        800.000000     4.700000   7498.000000
max    8000.00000       1000.000000     5.000000  10000.000000


In [3]:

data["Review"] = data["Review"].str.lower()
data["Review"] = data["Review"].replace(r'[^a-z0-9\s]', '', regex=True)
data['sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)
data= data.dropna()

In [4]:
max_features = 5000
max_length = 200

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])
X = pad_sequences(tokenizer.texts_to_sequences(
    data["Review"]), maxlen=max_length)
y = data['sentiment'].values



In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

In [12]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=16, input_length=max_length),
    SimpleRNN(64, activation='tanh', return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")

Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.7137 - loss: 0.6074 - val_accuracy: 0.7156 - val_loss: 0.5971
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7286 - loss: 0.5868 - val_accuracy: 0.7156 - val_loss: 0.5965
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - accuracy: 0.7144 - loss: 0.5982 - val_accuracy: 0.7156 - val_loss: 0.5970
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.7150 - loss: 0.5991 - val_accuracy: 0.7156 - val_loss: 0.5966
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.7105 - loss: 0.6002 - val_accuracy: 0.7156 - val_loss: 0.5964
Test accuracy: 0.72


In [14]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"


sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
Sentiment: Positive (Probability: 0.72)


In [15]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")

Epoch 1/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.7159 - loss: 0.5962 - val_accuracy: 0.7156 - val_loss: 0.5957
Epoch 2/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.7144 - loss: 0.5974 - val_accuracy: 0.7156 - val_loss: 0.5970
Epoch 3/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.7150 - loss: 0.5958 - val_accuracy: 0.7156 - val_loss: 0.5958
Epoch 4/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 0.7179 - loss: 0.5948 - val_accuracy: 0.7156 - val_loss: 0.5959
Epoch 5/5
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.7068 - loss: 0.6048 - val_accuracy: 0.7156 - val_loss: 0.5970
Test accuracy: 0.72


In [16]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"


sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Sentiment: Positive (Probability: 0.74)
